<a href="https://colab.research.google.com/github/francoishcm/BackTesting/blob/master/.iQuant_Investment_Programme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧰**INSTALL MODULES**

In [1]:
# Mount Google Drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install ta-lib v0.4.0
%%bash
wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
tar -xzf ta-lib-0.4.0-src.tar.gz
cd ta-lib/
./configure
make
make install

In [ ]:
# Install zipline
%pip install zipline-reloaded

In [ ]:
!pip install iso3166==2.0.2

In [ ]:
# Install Pyfolio
!pip install pyfolio-reloaded

In [ ]:
# Install matplot library
!pip install matplotlib

In [ ]:
# Install data bundle 'Quandl'
!pip install quandl

In [ ]:
import zipline
zipline.__version__

In [ ]:
import IPython
js_code = '''
document.querySelector("#output-area").appendChild(document.createTextNode("hello world!"));
'''
display(IPython.display.Javascript(js_code))

In [ ]:
!pip install nasdaq-data-link

# 💽**SET WORKING DIRECTORY**

In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!

# the base Google Drive directory
root_dir = "/content/drive/"


In [ ]:
import os 

# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!


# choose where you want your project files to be saved
project_folder = "MyDrive/Colab Notebooks/My Project Folder"


In [ ]:
def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

  # create a test file to make sure it shows up in the right place
  !touch 'new_file_in_working_directory.txt'
  print('\nYour working directory was changed to ' + root_dir + project_folder + \
        "\n\nAn empty text file was created there. You can also run !pwd to confirm the current working directory." )

create_and_set_working_directory(project_folder)

In [ ]:
# Confirm current working directory
!pwd

In [ ]:
import os
for f in os.listdir("//content/drive/MyDrive/Colab Notebooks/My Project Folder"):
	print(f)

# ⏳**INGEST DATA**

In [ ]:
# Ingest custom bundle
!zipline ingest --bundle 'crypto'

In [ ]:
# Ingest custom bundle
!zipline ingest --bundle 'equities_csvdir'

In [ ]:
# Ingest custom bundle
!zipline ingest --bundle 'random_futures_data'

In [ ]:
# Ingest custom bundle
!zipline ingest --bundle 'random_stock_data'

In [ ]:
'''
INGEST QUANDL STOCK BUNDLE
'''
# Ingest bundle API
!QUANDL_API_KEY=KUnssHvVERHb5XYu9C1- zipline ingest -b 'quandl'

In [ ]:
# Confirm existing bundles
!zipline bundles

In [ ]:
# clean everything older than <date>
!zipline clean -b crypto --after 2022-04-13

# 📊**ANDREAS CLENOW MODELS**

## **Clenow Momentum Model**

---


In [ ]:
#@title
%matplotlib inline

import zipline
from zipline import run_algorithm
from zipline.api import order_target_percent, symbol, set_commission, set_slippage, schedule_function, date_rules, time_rules
from pandas import Timestamp
from datetime import datetime
import pytz
import matplotlib.pyplot as plt 
import pyfolio as pf
import pandas as pd 
import numpy as np 
from scipy import stats
from zipline.finance.commission import PerDollar
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage

#Model Settings

intial_portfolio = 100000
momentum_window = 125
minimum_momentum = 40
portfolio_size = 30
vola_window = 20

#Commission and Slippage Settings

enable_commission = True 
commission_pct = 0.001 
enable_slippage = True 
slippage_volume_limit = 0.025
slippage_impact = 0.05

def momentum_score(ts):

  #Input: Price time series.Output: Annualized exponential regression slope, multiplied by the R2

  # Make a list of consecutive numbers 
  x = np.arange(len(ts))
  # Get logs
  log_ts = np.log(ts)
  # Calculate regression values
  slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts) 
  # Annualize percent
  annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100 
  #Adjust for fitness
  score = annualized_slope * (r_value ** 2) 
  return score

def volatility(ts):
  return ts.pct_change().rolling(vola_window).std().iloc[-1]

def output_progress(context):

  #Output some performance numbers during backtest run 
  #This code just prints out the past month's performance,
  # so that we have something to look at while the backtest runs.

  # Get today's date
  today = zipline.api.get_datetime().date()

  # Calculate percent difference since last month
  perf_pct = (context.portfolio.portfolio_value / context.last_month) - 1

  # Print performance, format as percent with two decimals. 
  print("{} - Last Month Result: {:.2%}".format(today, perf_pct))

  # Remember today's portfolio value for next month's calculation 
  context.last_month = context.portfolio.portfolio_value

#Initialization and trading logic


def initialize(context):

  # Set commission and slippage.
  if enable_commission:
    comm_model = PerDollar(cost=commission_pct) 
  else:
    comm_model = PerDollar(cost=0.0) 
  set_commission(comm_model)
  if enable_slippage: slippage_model=VolumeShareSlippage(volume_limit=slippage_volume_limit,
price_impact=slippage_impact) 
  else:
    slippage_model=FixedSlippage(spread=0.0) 
  set_slippage(slippage_model)

  # Used only for progress output. 
  context.last_month = intial_portfolio

  # Store index membership
  context.index_members = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/index members/sp500.csv', engine='python', error_bad_lines=False)


  #Schedule rebalance monthly. 
  schedule_function(
    func=rebalance, 
    date_rule=date_rules.month_start(), 
    time_rule=time_rules.market_open()
  )

def rebalance(context, data):
  # Write some progress output during the backtest 
  output_progress(context)

  # First, get today's date
  today = zipline.api.get_datetime().date()

  # Second, get the index makeup for all days prior to today.
  all_prior = context.index_members.loc[context.index_members.index < today]

  todays_universe = [ 
    symbol(ticker) for ticker in
    context.index_members.loc[context.index_members.index < today].iloc[-1,0].split(',')
  ]

  # Get historical data
  hist = data.history(todays_universe, "close", momentum_window, "1d")

  # Make momentum ranking table
  ranking_table = hist.apply(momentum_score).sort_values(ascending=False)

  #Sell Logic
  #First we check if any existing position should be sold.
    #Sell if stock is no longer part of index.
    #Sell if stock has too low momentum value.

  kept_positions = list(context.portfolio.positions.keys()) 
  for security in context.portfolio.positions:
    if (security not in todays_universe): 
      order_target_percent(security, 0.0) 
      kept_positions.remove(security)
    elif ranking_table[security] < minimum_momentum: 
      order_target_percent(security, 0.0) 
      kept_positions.remove(security)

  #Stock Selection Logic
  #Check how many stocks we are keeping from last month.
  #Fill from top of ranking list, until we reach the desired total number of portfolio holdings.

  replacement_stocks = portfolio_size - len(kept_positions) 
  buy_list = ranking_table.loc[
    ~ranking_table.index.isin(kept_positions)][:replacement_stocks]
  new_portfolio = pd.concat(
    (buy_list,
    ranking_table.loc[ranking_table.index.isin(kept_positions)])
  )
  buy_list = ranking_table.loc[
    ~ranking_table.index.isin(kept_positions)][:replacement_stocks]

  #Calculate inverse volatility for stocks, and make target position weights.

  vola_table = hist[new_portfolio.index].apply(volatility) 
  inv_vola_table = 1 / vola_table
  sum_inv_vola = np.sum(inv_vola_table) 
  vola_target_weights = inv_vola_table / sum_inv_vola
  for security, rank in new_portfolio.iteritems(): 
    weight = vola_target_weights[security]
    if security in kept_positions:
      order_target_percent(security, weight)
    else:
      if ranking_table[security] > minimum_momentum: 
        order_target_percent(security, weight)

def analyze(context, perf):
  perf['max'] = perf.portfolio_value.cummax() 
  perf['dd'] = (perf.portfolio_value / perf['max']) - 1 
  maxdd = perf['dd'].min()

  ann_ret = (np.power((perf.portfolio_value.iloc[-1] / perf.portfolio_value.iloc[0]),(252 / len(perf)))) - 1

  print("Annualized Return: {:.2%} Max Drawdown: {:.2%}".format(ann_ret, maxdd))

  return
'''
start = datetime(1997, 1, 1, 8, 15, 12, 0, pytz.UTC)
end = datetime(2018, 12, 31, 8, 15, 12, 0, pytz.UTC)
'''

start = pd.Timestamp('1997-1-1', tz='utc')
end = pd.Timestamp('2018-12-31', tz='utc')

perf = zipline.run_algorithm(start=start, end=end, 
                             initialize=initialize, 
                             analyze=analyze, 
                             capital_base=intial_portfolio, 
                             data_frequency = 'daily', 
                             bundle='quandl' )

## **Clenow Trend Model**

In [ ]:

%matplotlib inline

import zipline
from zipline.api import future_symbol,  \
    set_commission, set_slippage, schedule_function, date_rules, \
    time_rules, continuous_future, order_target
from pandas import Timestamp
import pytz
import datetime as datetime
import matplotlib.pyplot as plt
import pyfolio as pf
import pandas as pd
import numpy as np  
from zipline.finance.commission import PerTrade, PerContract
from zipline.finance.slippage import VolumeShareSlippage, \
    FixedSlippage, VolatilityVolumeShare

# These lines are for the dynamic text reporting
from IPython.display import display
import ipywidgets as widgets
out = widgets.HTML()
display(out)

"""
Model Settings
"""
starting_portfolio = 50000000
risk_factor = 0.0015
stop_distance = 3
breakout_window = 50
vola_window = 40
slow_ma = 80
fast_ma = 40
enable_commission = True
enable_slippage = True  


def report_result(context, data):
    context.months += 1
    today = zipline.api.get_datetime().date()
    # Calculate annualized return so far
    ann_ret = np.power(context.portfolio.portfolio_value / starting_portfolio, 
                   12 / context.months) - 1
    
    # Update the text
    out.value = """{} We have traded <b>{}</b> months 
    and the annualized return is <b>{:.2%}</b>""".format(today, context.months, ann_ret)

def roll_futures(context, data):
    open_orders = zipline.api.get_open_orders()
    
    for held_contract in context.portfolio.positions:
        # don't roll positions that are set to change by core logic
        if held_contract in open_orders: 
            continue
        
        # Save some time by only checking rolls for
        # contracts stopping trading in the next days
        days_to_auto_close = (
            held_contract.auto_close_date.date() - data.current_session.date()
        ).days
        if days_to_auto_close > 5:
            continue        
        
        # Make a continuation
        continuation = continuous_future(
                held_contract.root_symbol, 
                offset=0, 
                roll='volume', 
                adjustment='mul'
                )
        
        # Get the current contract of the continuation
        continuation_contract = data.current(continuation, 'contract')
        
        if continuation_contract != held_contract:
            # Check how many contracts we hold
            pos_size = context.portfolio.positions[held_contract].amount         
            # Close current position
            order_target(held_contract, 0)
            # Open new position
            order_target(continuation_contract, pos_size)     
            
def position_size(portfolio_value, std, point_value):
    target_variation = portfolio_value * risk_factor
    contract_variation = std * point_value
    contracts = target_variation / contract_variation
    return int(np.nan_to_num(contracts)) 
    
def initialize(context):
    
    """
    Cost Settings
    """
    if enable_commission:
        comm_model = PerContract(cost=0.85, exchange_fee=1.5)
    else:
        comm_model = PerTrade(cost=0.0)
        
    set_commission(us_futures=comm_model)
    
    if enable_slippage:
        slippage_model=VolatilityVolumeShare(volume_limit=0.2)
    else:
        slippage_model=FixedSlippage(spread=0.0)      
        
    set_slippage(us_futures=slippage_model)
    
    """
    Markets to trade
    """ 
    currencies = [
        'AD',
        'BP',
        'CD',
        'CU',
        'DX',
        'JY',
        'NE',
        'SF',
    ]
    
    agricultural = [
        '_C',
        'CT',
        'FC',
        'KC',
        'LR',
        'LS',
        '_O',
        '_S',
        'SB',
        'SM',
        '_W',
    ]
    nonagricultural = [
        'CL',
        'GC',
        'HG',
        'HO',
        'LG',
        'NG',
        'PA',
        'PL',
        'RB',
        'SI',
    ]
    equities = [
        'ES',
        'NK',
        'NQ',
        'TW',
        'VX',
        'YM',
    ]
    rates = [
        'ED',
        'FV',
        'TU',
        'TY',
        'US',
    ]
    
    # Make a list of all the markets
    markets = currencies + agricultural + nonagricultural + equities + rates
    
    # Make a list of all continuations
    context.universe = [
        continuous_future(market, offset=0, roll='volume', adjustment='mul')
            for market in markets
    ]
    
    # We'll use these to keep track of best position reading
    # Used to calculate stop points.
    context.highest_in_position = {market: 0 for market in markets} 
    context.lowest_in_position = {market: 0 for market in markets}    
    
    # Schedule the daily trading
    schedule_function(daily_trade, date_rules.every_day(), time_rules.market_close())
    
    # We'll just use this for the progress output
    # during the backtest. Doesn't impact anything.
    context.months = 0    
    
    # Schedule monthly report output
    schedule_function(
        func=report_result,
        date_rule=date_rules.month_start(),
        time_rule=time_rules.market_open()
    ) 
    
def analyze(context, perf):
    returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)
    pf.create_returns_tear_sheet(returns, benchmark_rets=None)
    
def daily_trade(context, data):
    # Get continuation data
    hist = data.history(
        context.universe, 
        fields=['close','volume'], 
        frequency='1d', 
        bar_count=250,
    )
    
    # Calculate trend
    hist['trend'] = hist['close'].ewm(span=fast_ma).mean() > hist['close'].ewm(span=slow_ma).mean()    
    
    # Make dictionary of open positions
    open_pos = {
        pos.root_symbol: pos 
        for pos in context.portfolio.positions
    } 
    
    # Iterate markets, check for trades
    for continuation in context.universe:
        
        # Get root symbol of continuation
        root = continuation.root_symbol
        
        # Slice off history for just this market
        h = hist.xs(continuation, 2)
        
        # Get standard deviation
        std = h.close.diff()[-vola_window:].std()

        if root in open_pos: # Position is open

            # Get position
            p = context.portfolio.positions[open_pos[root]]
            
            if p.amount > 0: # Position is long
                if context.highest_in_position[root] == 0: # First day holding the position
                    context.highest_in_position[root] = p.cost_basis
                else:
                    context.highest_in_position[root] = max(
                        h['close'].iloc[-1], context.highest_in_position[root]
                    ) 
                    
                # Calculate stop point
                stop = context.highest_in_position[root] - (std  * stop_distance)
                # Check if stop is hit
                if h.iloc[-1]['close'] < stop:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.highest_in_position[root] = 0
                # Check if trend has flipped
                elif h['trend'].iloc[-1] == False:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.highest_in_position[root] = 0
                    
            else: # Position is short
                if context.lowest_in_position[root] == 0: # First day holding the position
                    context.lowest_in_position[root] = p.cost_basis
                else:
                    context.lowest_in_position[root] = min(
                        h['close'].iloc[-1], context.lowest_in_position[root]
                    )
                
                # Calculate stop point
                stop = context.lowest_in_position[root] + (std  * stop_distance)
                
                # Check if stop is hit
                if h.iloc[-1]['close'] > stop:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.lowest_in_position[root] = 0
                # Check if trend has flipped
                elif h['trend'].iloc[-1] == True:
                    contract = open_pos[root]
                    order_target(contract, 0)
                    context.lowest_in_position[root] = 0                         
        
        else: # No position on
            if h['trend'].iloc[-1]: # Bull trend
                # Check if we just made a new high
                if h['close'][-1] == h[-breakout_window:]['close'].max(): 
                    contract = data.current(continuation, 'contract')

                    contracts_to_trade = position_size( \
                                                       context.portfolio.portfolio_value, \
                                                       std, \
                                                       contract.price_multiplier)
                    
                    # Limit size to 20% of avg. daily volume
                    contracts_cap = int(h['volume'][-20:].mean() * 0.2)
                    contracts_to_trade = min(contracts_to_trade, contracts_cap)
                    
                    # Place the order
                    order_target(contract, contracts_to_trade)
             
            else: # Bear trend
                # Check if we just made a new low
                if h['close'][-1] == h[-breakout_window:]['close'].min(): 
                    contract = data.current(continuation, 'contract')

                    contracts_to_trade = position_size( \
                                                       context.portfolio.portfolio_value, \
                                                       std, \
                                                       contract.price_multiplier)
                    
                    # Limit size to 20% of avg. daily volume
                    contracts_cap = int(h['volume'][-20:].mean() * 0.2)
                    contracts_to_trade = min(contracts_to_trade, contracts_cap)
                    
                    # Place the order
                    order_target(contract, -1 * contracts_to_trade)
    
    # If we have open positions, check for rolls
    if len(open_pos) > 0:   
        roll_futures(context, data)                
                        

start = pd.Timestamp('2003-01-01', tz='utc')
end = pd.Timestamp('2017-12-31', tz='utc')

perf = zipline.run_algorithm(
    start=start, end=end, 
    initialize=initialize, 
    analyze=analyze,
    capital_base=starting_portfolio,  
    data_frequency = 'daily', 
    bundle='random_futures_data' ) 




### 💻**MODEL TESTING**

#**DIGITAL ASSETS PROGRAMME**



## **Systems**

---



### Single Asset Backtest

In [ ]:
# This ensures that our graphs will be shown properly in the notebook.
%matplotlib inline

# Import libraries
import zipline
from zipline import run_algorithm
from zipline.api import order_target_percent, symbol
from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import pandas as pd

def initialize(context):
  # Which asset to trade
  context.asset = symbol('btc')
  
  # Moving average window
  context.index_average_window = 100

def handle_data(context, data):
  # Request history for the stock
  btc_hist = data.history(context.asset, "close",
                               context.index_average_window, "1d")
  
  # Check if price is above moving average
  if btc_hist[-1] > btc_hist.mean():
    asset_weight = 1.0
  else:
    asset_weight = 0.0
    # Place order
    order_target_percent(context.asset, asset_weight)

def analyze(context, perf):
  
  fig = plt.figure(figsize=(12, 8))
  
    # First char
  ax = fig.add_subplot(311)
  ax.set_title('Strategy Results')
  ax.semilogy(perf['portfolio_value'], linestyle='-',
              label='Equity Curve', linewidth=3.0)
  ax.legend()
  ax.grid(False)

  # Second chart
  ax = fig.add_subplot(312)
  ax.plot(perf['gross_leverage'],
  label='Exposure', linestyle='-', linewidth=1.0)
  ax.legend()
  ax.grid(True)

  # Third chart
  ax = fig.add_subplot(313)
  ax.plot(perf['returns'], label='Returns', linestyle='-.',
          linewidth=1.0)

### Equal Weight Model

In [ ]:
# This ensures that our graphs will be shown properly in the notebook.
%matplotlib inline

# Import a few libraries we need
from zipline import run_algorithm
from zipline.api import order_target_percent, record, symbol, set_benchmark
import pyfolio as pf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def initialize(context):

  # Which stock to trade

  """
  dji = ["JEC","BBY","MSFT","MCHP","PEP",
         "RAD","PTC","GCO","FAST","CTL",
         "APA","EL","TMK","VVI","HPQ",
         "CMCSA","JCI","T"]

  dji = ["aave","ada","algo","alpha","ant",
         "bal","bat","bch","bnb","bsv",
         "btc"]
  """

  dji = ["btc"]


  # Make a list of symbols from the list of tickers
  context.dji_symbols = [symbol(s) for s in dji]

  # Moving average window
  context.index_average_window = 52
  
def handle_data(context, data):
  
  # Get history for all the stocks
  stock_hist = data.history(context.dji_symbols, "close", context.index_average_window, "1d")

  # Make an empty DataFrame to start with
  stock_analytics = pd.DataFrame()

  # Add column for above or below average
  stock_analytics['above_mean'] = stock_hist.iloc[-1] > stock_hist.mean()

  # Set weight for stocks to buy
  stock_analytics.loc[stock_analytics['above_mean'] == True, 'weight'] = 1/len(context.dji_symbols)

  # Set weight to zero for the rest
  stock_analytics.loc[stock_analytics['above_mean'] == False, 'weight'] = 0.0

  # Iterate each row and place trades
  for stock, analytics in stock_analytics.iterrows():

    # Check if the stock can be traded
    if data.can_trade(stock):

      # Place the trade
      order_target_percent(stock, analytics['weight'])

def analyze(context, perf):

  # Use PyFolio to generate a performance report
  returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)

  benchmark_period_return = perf['benchmark_period_return']

  daily_benchmark_returns = np.exp(np.log(benchmark_period_return + 1.0).diff()) - 1

  # Create tear sheet
  pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions, benchmark_rets=None)

"""

def analyze(context, perf):

  fig = plt.figure(figsize=(12, 8))

  # First chart
  ax = fig.add_subplot(311)
  ax.set_title('Strategy Results')
  ax.plot(perf['portfolio_value'], linestyle='-',
          label='Equity Curve', linewidth=3.0)
  ax.legend()
  ax.grid(False)

  # Second chart
  ax = fig.add_subplot(312)
  ax.plot(perf['gross_leverage'],label='Exposure',
          linestyle='-', linewidth=1.0)
  ax.legend()
  ax.grid(True)
  
  # Third chart
  ax = fig.add_subplot(313)
  ax.plot(perf['returns'], label='Returns', linestyle='-.',
          linewidth=1.0)
  ax.legend()
  ax.grid(True)

  """

# Set start and end date
start_date = pd.Timestamp('2016-05-02', tz='utc')
end_date = pd.Timestamp('2020-05-02', tz='utc')

# Fire off the backtest
perf = run_algorithm(
start=start_date,
end=end_date,
initialize=initialize,
analyze=analyze,
handle_data=handle_data,
capital_base=10000,
data_frequency = 'daily', 
bundle= 'crypto',)

In [ ]:
#Export performance results to disk in csv file

perf.portfolio_value.to_csv('ewm_momentum_model.csv')

### Momentum Model

In [ ]:
#@title

%matplotlib inline

import zipline
from zipline import run_algorithm
from zipline.api import order_target_percent, symbol, set_commission, set_slippage, schedule_function, date_rules, time_rules
import matplotlib.pyplot as plt 
import pyfolio as pf
import pandas as pd 
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
from zipline.finance.commission import PerDollar
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage


initial_portfolio = 10000
minimum_momentum = 10
portfolio_size = 1
vola_window = 30

def momentum_score(ts):

  #Input: Price time series.Output: Annualized exponential regression slope, multiplied by the R2

  # Make a list of consecutive numbers 
  x = np.arange(len(ts))
  # Get logs
  log_ts = np.log(ts)
  # Calculate regression values
  slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts) 
  # Annualize percent
  annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100 
  #Adjust for fitness
  score = annualized_slope * (r_value ** 2) 
  return score

def volatility(ts):
  return ts.pct_change().rolling(vola_window).std().iloc[-1]


#Initialization and trading logic


def initialize(context):

  context.rolling_window = 100

  #Commission and Slippage Settings

  enable_commission = True 
  commission_pct = 0.001 
  enable_slippage = True 
  slippage_volume_limit = 0.025
  slippage_impact = 0.05

  """
  dji = ["AAVE","ADA","ALPHA","BCH","BTC",
          "DOGE","DOT","ETH","LTC","USDT",
          "XLM","XMR","XRP"]
  """
  dji = ["btc","ltc","eth"]

  
  # Make a list of symbols from the list of tickers
  context.dji_symbols = [symbol(s) for s in dji]

  # Set commission and slippage.
  if enable_commission:
    comm_model = PerDollar(cost=commission_pct) 
  else:
    comm_model = PerDollar(cost=0.0) 
  set_commission(comm_model)
  if enable_slippage: slippage_model=VolumeShareSlippage(volume_limit=slippage_volume_limit,
price_impact=slippage_impact) 
  else:
    slippage_model=FixedSlippage(spread=0.0) 
  set_slippage(slippage_model)


  #Schedule rebalance monthly. 
  schedule_function(
    func=rebalance, 
    date_rule=date_rules.month_start(), 
    time_rule=time_rules.market_open()
  )

def rebalance(context, data):

  # Get historical data
  hist = data.history(context.dji_symbols, "close", context.rolling_window, "1d")

  # Make momentum ranking table
  ranking_table = hist.apply(momentum_score).sort_values(ascending=False)

  #Sell Logic
  #First we check if any existing position should be sold.
    #Sell if stock is no longer part of index.
    #Sell if stock has too low momentum value.

  kept_positions = list(context.portfolio.positions.keys()) 
  for security in context.portfolio.positions:
    if ranking_table[security] < minimum_momentum: 
      order_target_percent(security, 0.0)
      kept_positions.remove(security)
 
  #Stock Selection Logic
  #Check how many stocks we are keeping from last month.
  #Fill from top of ranking list, until we reach the desired total number of portfolio holdings.

  replacement_stocks = portfolio_size - len(kept_positions) 
  buy_list = ranking_table.loc[
    ~ranking_table.index.isin(kept_positions)][:replacement_stocks]
  new_portfolio = pd.concat(
    (buy_list,
    ranking_table.loc[ranking_table.index.isin(kept_positions)])
  )
  buy_list = ranking_table.loc[
    ~ranking_table.index.isin(kept_positions)][:replacement_stocks]

  #Calculate inverse volatility for stocks, and make target position weights.

  vola_table = hist[new_portfolio.index].apply(volatility) 
  inv_vola_table = 1 / vola_table
  sum_inv_vola = np.sum(inv_vola_table) 
  vola_target_weights = inv_vola_table / sum_inv_vola
  for security, rank in new_portfolio.iteritems(): 
    weight = vola_target_weights[security]
    if security in kept_positions:
      order_target_percent(security, weight)
    else:
      if ranking_table[security] > minimum_momentum: 
        order_target_percent(security, weight)


def analyze(context, perf):
    
  # Use PyFolio to generate a performance report
  returns, positions, transactions = pf.utils.extract_rets_pos_txn_from_zipline(perf)

  benchmark_period_return = perf['benchmark_period_return']

  daily_benchmark_returns = np.exp(np.log(benchmark_period_return + 1.0).diff()) - 1

  # Create tear sheet
  pf.create_full_tear_sheet(returns, positions=positions, transactions=transactions, benchmark_rets=None)

start= pd.Timestamp('2015-5-1', tz='utc')
end = pd.Timestamp('2022-5-2', tz='utc')

perf = zipline.run_algorithm(start=start, 
                             end=end, 
                             initialize=initialize, 
                             analyze=analyze, 
                             capital_base=initial_portfolio, 
                             data_frequency='daily',
                             bundle='crypto')

In [ ]:
#Export Performance Result to disk in csv file

perf.portfolio_value.to_csv('crypto_momentum.csv')

### **Model Performance**

# **DATA CLEANING**



## Futures

In [ ]:
# Import modules
import nasdaqdatalink
import quandl
# Get the data for Futures, Continuous Contract #6.
import matplotlib.pyplot as plt
data = quandl.get("CHRIS/CME_YM1",start_date="2017-12-03", end_date="2018-12-03", api_key='WDDHaLh3eG6vrEgiCCqy')

In [ ]:
# Plot the prices
data.Settle.plot()
plt.show()

In [ ]:
#import required libraries
import pandas as pd
from datetime import datetime
import numpy as np

data = data.rename(columns={'Open': 'open','High':'high',
                                'Low':'low','Last':'close',
                                'Volume':'volume','Previous Day Open Interest': 'openinterest'}, index=None)

data['expiration_date'] = '2018-12-21'
data['root_symbol'] = 'YM'
data['symbol'] = 'YMZ18'

new_data = data.drop(['Change', 'Settle',], axis=1)

print(new_data.head())



In [ ]:
new_data.to_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/Futures/YMZ18.csv')

## Digital assets

### Adjusting model performace csv data for comparison

In [ ]:
import pandas as pd
import numpy as np

A = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/Backtests/Raw Model Performance Data/Digital Assets/crypto_momentum.csv', parse_dates=True, index_col=0)

A.index = A.index.strftime('%Y/%m/%d')

header_row = 1

A.columns = A.iloc[header_row]

A

In [ ]:
A.info()

In [ ]:

A.to_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/Backtests/crypto_momentum_2.csv')


### Adjusting crypto indices csv data for comparison

In [ ]:
#import required libraries
import pandas as pd
from datetime import datetime
import exchange_calendars as xcals
from zipline import get_calendar
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/indices/SPCBDM.csv', encoding='cp1252')
df.info()
print(df)

In [ ]:

df.drop([1355],axis=0, inplace = True )

#Convert 'date' Column to datetime
df['2/28/2017'] = pd.to_datetime(df['2/28/2017'], utc=True)

#Set 'date' column as index
df.set_index('2/28/2017', inplace=True)

# Get all expected trading sessions in new dataframe.
sessions = get_calendar('NYSE').sessions_in_range('2017-02-28', '2022-05-05')

df.index = df.index.strftime('%Y/%m/%d')

header_row = 1

df.columns = df.iloc[header_row]

df.tail(5)
df.info()
df

In [ ]:
df = df[~df.index.duplicated()]
df

In [ ]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/Backtests/SPCBDM_test3.csv')

### Removing weekends from timeseries

In [ ]:
#@title
#Import statements
%matplotlib inline
import pandas as pd
import datetime as dt
import numpy as np

#Load csv file from disc
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto_data/DOGE.csv')

#Set date column to datetime
df['time'] = pd.to_datetime(df['time'], errors='coerce')

#Remove weekends from data
df = df[df.time.dt.weekday < 5]

#Set 'date' columns as Index
df.set_index("time", inplace = True)

#display weekday dataframe
df


In [ ]:
#inspect dataframe
df.info()

In [ ]:
#Save dataframe to csv

df.to_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto weekday data/DODGE_weekday.csv')

In [ ]:
#Load final data to file to dataframe
dodge = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/crypto weekday data/DODGE_weekday.csv')

print(dodge)



In [ ]:
#Remove weekends from dataframe
btc = usholidays[usholidays.date.dt.weekday < 5]

#Set 'date' column as index
btc.set_index('date', inplace = True)

btc

In [ ]:
#import US holiday calender
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

#Set period between start and end date in dataframe to identify and remove holidays
holidays = calendar().holidays(start='2009-01-05', end='2022-05-24') 
m = raw_data['date'].isin(holidays)
usholidays = raw_data[~m].copy()

#print new dataframe with holidays removed
usholidays

### Adjusting crypto data from **Yahoo Finance** for testing with NYSE trading calender.

In [ ]:
#import required libraries
import pandas as pd
from datetime import datetime
import exchange_calendars as xcals
from zipline import get_calendar
import numpy as np

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/yahoo_data/yahoo raw data/BTC-USD.csv', parse_dates=True, index_col=0)

# Remove Adj Close Column
new_df = df.drop(['Adj Close'], axis=1)

new_df["Volume"] = new_df["Volume"].astype(float)

#Reset index
new_df.reset_index(inplace = True)

#Convert 'date' Column to datetime
new_df['Date'] = pd.to_datetime(new_df['Date'], utc=True)

#Set 'date' column as index
new_df.set_index('Date', inplace=True)

# Get all expected trading sessions in new dataframe.
sessions = get_calendar('NYSE').sessions_in_range('2009-01-05', '2022-07-07')

# To set the trading session in  new dataframe to the NYSE Calender
btc = new_df.reindex(sessions)

#Reset index again to change the index name
btc.reset_index(inplace = True)

#Rename index column to 'date'
btc = btc.rename(columns={'index': 'trading_date'}, index=None)

#Change date format to Year-Month-Day
btc['trading_date'] =  pd.to_datetime(btc['trading_date']).dt.strftime('%Y-%m-%d')

crypto = btc.dropna()

#Change 'PriceUSD' columns to 'close'
crypto.rename(columns = {'Open':'open',
                      'High':'high',
                      'Low':'low',
                      'Close':'close',
                      'Volume':'volume'}, inplace = True)

crypto.set_index('trading_date', inplace=True)

crypto.info()

crypto.to_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/yahoo_data/yahoo cleaned data/btc.csv')

crypto

### Adjusting crypto data from **Coinmetrics** for testing with NYSE trading calender.

In [ ]:
#import required libraries
import pandas as pd
from datetime import datetime
import exchange_calendars as xcals
from zipline import get_calendar
import numpy as np

#load raw data from file
raw_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/coinmetrics_data/btc.csv', parse_dates=True, index_col=0)

new = raw_data.drop(raw_data.loc[:, :'PriceBTC'].columns, axis=1)


#drop columns not needed. Only closing prices required
new.drop(new.iloc[:, 1:76], inplace = True, axis = 1)

#Add new columns
new['high'] =0
new['low'] =0
new['open'] =0
new['volume'] =0

# Rearrange columns to set timestamp as first column
new = new[['high','open','low','PriceUSD','volume']]

#Change 'PriceUSD' columns to 'close'
new.rename(columns = {'PriceUSD':'close'}, inplace = True)

"""
#Drop row (date) not needed
new=new.drop(['2022-05-25'])
"""

#Fill 'close' column with integer values '0'
new["close"].fillna(0, inplace=True)

#Reset index
new.reset_index(inplace = True)

#Rename time columns to date
new = new.rename(columns={'time': 'date'}, index=None)

#Drop all NaN values from dataframe
new.dropna()

#Convert columns datatype from integer to float
new["high"] = new["high"].astype(float)
new["open"] = new["open"].astype(float)
new["low"] = new["low"].astype(float)
new["volume"] = new["volume"].astype(float)

#Convert 'date' Column to datetime
new['date'] = pd.to_datetime(new['date'], utc=True)

#Set 'date' column as index
new.set_index('date', inplace=True)

# Get all expected trading sessions in new dataframe.
sessions = get_calendar('NYSE').sessions_in_range('2009-01-05', '2022-05-24')

# To set the trading session in  new dataframe to the NYSE Calender
btc = new.reindex(sessions)

#Reset index again to change the index name
btc.reset_index(inplace = True)

#Rename index column to 'date'
btc = btc.rename(columns={'index': 'date'}, index=None)

#Change date format to Year-Month-Day
btc['date'] =  pd.to_datetime(btc['date']).dt.strftime('%Y-%m-%d')

"""
btc.set_index('date', inplace=True)
"""

# Remove two columns name is 'C' and 'D'
df = btc.drop(['high', 'open','low'], axis=1)

df['open'] = df['close'] - (0 * df ['close'])
df['high'] = df['open'] - (0 * df ['open'])
df['low'] = df['high'] - (0 * df ['high'])

# Rearrange columns
df = df[['date','open','high','low','close','volume']]

df.set_index('date', inplace=True)

crypto = df.dropna()

crypto.to_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/crypto/final crypto bundle/btc.csv')



## Exploratory Data Analyses (EDA)





In [ ]:
!pip install mpld3

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sn
import matplotlib.pyplot as plt
"""
import mpld3
mpld3.enable_notebook()
"""
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 2)

In [ ]:
B = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/My Project Folder/random_futures/YMZ18.csv', parse_dates=True, index_col=0)

B.info()

print(B.head(5))

In [ ]:
new_data.info()

print(new_data.head(5))

In [ ]:
pd.set_option('display.max_rows', None)

B.describe()

In [ ]:
pd.set_option('display.max_rows', None)

new_data.describe()

In [ ]:
print(B)
print(new_data)

In [ ]:
"""
B['Open'].plot(figsize=(12,6), 
                  linestyle='--',color='black',
                  legend='Open')

B['High'].plot(figsize=(12,6),
                   linestyle='-',color='grey',
                   legend='High')
"""
B['close'].plot(figsize=(12,6),
                 linestyle=':',color='black',
                 legend='Low')

new_data['close'].plot(figsize=(12,6),
                  linestyle='-',color='black',
                  legend='Close')


In [ ]:
no_outlier_prices = B[(np.abs(stats.zscore(B)) <
6).all(axis=1)]

In [ ]:
no_outlier_prices['100'].plot(figsize=(12,6), linestyle='--',
color='black', legend='Close')

In [ ]:
pd.set_option('display.max_rows', None)

no_outlier_prices[['533.95']].describe()

# **MARKETSTACK & STOCKDIO**

# Construct list of stock tickers from investment universe

In [ ]:
import pandas as pd
import json
import requests
params = {'access_key': 'e52cf3b93696352e880916f8c8adbf0c',
          'limit': 350}
api_result = requests.get('http://api.marketstack.com/v1/exchanges/XJSE/tickers', params)
api_response = api_result.json()
print(f"Exchange Name = {api_response['data']['name']}")
for ticker in api_response['data']['tickers']:
  print(f"{ticker['name']}: {ticker['symbol']}")

Exchange Name = Johannesburg Stock Exchange
PROSUS N.V.: PRX.XJSE
ANHEUSER-BUSCH INBEV SA NV: ANH.XJSE
BHP GROUP PLC: BHP.XJSE
BRITISH AMERICAN TOB PLC: BTI.XJSE
NASPERS LTD -N-: NPN.XJSE
COMPAGNIE FIN RICHEMONT: CFR.XJSE
GLENCORE PLC: GLN.XJSE
ANGLO AMERICAN PLC: AGL.XJSE
FIRSTRAND LTD: FSR.XJSE
ANGLO AMERICAN PLAT LTD: AMS.XJSE
STANDARD BANK GROUP LTD: SBK.XJSE
VODACOM GROUP LTD: VOD.XJSE
MONDI PLC: MNP.XJSE
CAPITEC BANK HLDGS LTD: CPI.XJSE
BEE - SASOL LIMITED: SOLBE1.XJSE
SANLAM LIMITED: SLM.XJSE
MTN GROUP LTD: MTN.XJSE
ANGLOGOLD ASHANTI LTD: ANG.XJSE
SASOL LIMITED: SOL.XJSE
ABSA GROUP LIMITED: ABG.XJSE
SOUTH32 LIMITED: S32.XJSE
RMB HOLDINGS LTD: RMH.XJSE
IMPALA PLATINUM HLGS LTD: IMP.XJSE
BID CORPORATION LTD: BID.XJSE
REMGRO LTD: REM.XJSE
KUMBA IRON ORE LTD: KIO.XJSE
NEDBANK GROUP LTD: NED.XJSE
SIBANYE STILLWATER LTD: SSW.XJSE
GOLD FIELDS LTD: GFI.XJSE
INVESTEC LTD: INL.XJSE
INVESTEC PLC: INP.XJSE
OLD MUTUAL LIMITED: OMU.XJSE
NEPI ROCKCASTLE PLC: NRP.XJSE
DISCOVERY LTD: DSY.XJSE
SH

In [ ]:
import json

# Serializing json
json_object = json.dumps(api_response, indent=4)

# Writing to sample.json
with open("jse.json", "w") as outfile:
    outfile.write(json_object)

In [ ]:
# Opening JSON file
with open('jse.json', 'r') as openfile:
 
    # Reading from json file
    json_object = json.load(openfile)
 
print(json_object)
print(type(json_object))

{'pagination': {'limit': 350, 'offset': 0, 'count': 350, 'total': 534}, 'data': {'name': 'Johannesburg Stock Exchange', 'acronym': 'JSE', 'mic': 'XJSE', 'country': 'South Africa', 'city': 'Johannesburg', 'website': 'WWW.JSE.CO.ZA', 'tickers': [{'name': 'PROSUS N.V.', 'symbol': 'PRX.XJSE', 'has_intraday': False, 'has_eod': True}, {'name': 'ANHEUSER-BUSCH INBEV SA NV', 'symbol': 'ANH.XJSE', 'has_intraday': False, 'has_eod': True}, {'name': 'BHP GROUP PLC', 'symbol': 'BHP.XJSE', 'has_intraday': False, 'has_eod': True}, {'name': 'BRITISH AMERICAN TOB PLC', 'symbol': 'BTI.XJSE', 'has_intraday': False, 'has_eod': True}, {'name': 'NASPERS LTD -N-', 'symbol': 'NPN.XJSE', 'has_intraday': False, 'has_eod': True}, {'name': 'COMPAGNIE FIN RICHEMONT', 'symbol': 'CFR.XJSE', 'has_intraday': False, 'has_eod': True}, {'name': 'GLENCORE PLC', 'symbol': 'GLN.XJSE', 'has_intraday': False, 'has_eod': True}, {'name': 'ANGLO AMERICAN PLC', 'symbol': 'AGL.XJSE', 'has_intraday': False, 'has_eod': True}, {'name

In [ ]:
print(json_object.keys())

In [ ]:
print(json_object["data"].keys())

In [ ]:
data = pd.json_normalize(json_object["data"]["tickers"])

print(data)

                                                  name       symbol  \
0                                          PROSUS N.V.     PRX.XJSE   
1                           ANHEUSER-BUSCH INBEV SA NV     ANH.XJSE   
2                                        BHP GROUP PLC     BHP.XJSE   
3                             BRITISH AMERICAN TOB PLC     BTI.XJSE   
4                                      NASPERS LTD -N-     NPN.XJSE   
..                                                 ...          ...   
345                            4Sight Holdings Limited     4SI.XJSE   
346                              Arden Capital Limited     ACZ.XJSE   
347                                    AH-Vest Limited     AHL.XJSE   
348                    Cloud Atlas AMI Big50 ex-SA ETF  AMIB50.XJSE   
349  Cloud Atlas (RF) Proprietary Limited - AMI Rea...   AMIRE.XJSE   

     has_intraday  has_eod  
0           False     True  
1           False     True  
2           False     True  
3           False     True  
4 

In [ ]:
'''
# use the pandas module
with open('sample.json') as json_file:
    jse = pd.read_json(json_file, orient='index')

print(jse)
'''

In [ ]:
# dropping null value columns to avoid errors
data.dropna(inplace = True)
 
# new data frame with split value columns
new = data["symbol"].str.split(".", n = 1, expand = True)
 
# making separate first name column from new data frame
data["ticker"]= new[0]
 
# Dropping old Name columns
data.drop(columns =["has_intraday","has_eod","symbol"], inplace = True)
 
# df display
data

,name,ticker
0,PROSUS N.V.,PRX
1,ANHEUSER-BUSCH INBEV SA NV,ANH
2,BHP GROUP PLC,BHP
3,BRITISH AMERICAN TOB PLC,BTI
4,NASPERS LTD -N-,NPN
...,...,...
345,4Sight Holdings Limited,4SI
346,Arden Capital Limited,ACZ
347,AH-Vest Limited,AHL
348,Cloud Atlas AMI Big50 ex-SA ETF,AMIB50


In [ ]:
# Converting a specific Dataframe 
# column to list using Series.tolist()
Stocks = data["ticker"].tolist()
  
print("Converting ticker to list:")
  
# displaying list
Stocks

Converting ticker to list:


['PRX',
 'ANH',
 'BHP',
 'BTI',
 'NPN',
 'CFR',
 'GLN',
 'AGL',
 'FSR',
 'AMS',
 'SBK',
 'VOD',
 'MNP',
 'CPI',
 'SOLBE1',
 'SLM',
 'MTN',
 'ANG',
 'SOL',
 'ABG',
 'S32',
 'RMH',
 'IMP',
 'BID',
 'REM',
 'KIO',
 'NED',
 'SSW',
 'GFI',
 'INL',
 'INP',
 'OMU',
 'NRP',
 'DSY',
 'SHP',
 'BVT',
 'RNI',
 'CLS',
 'PPH',
 'QLT',
 'GRT',
 'MEI',
 'APN',
 'PSG',
 'MCG',
 'MRP',
 'NHM',
 'RMI',
 'FFA',
 'WHL',
 'CCO',
 'LHC',
 'HMN',
 'SPP',
 'HAR',
 'RDF',
 'TFG',
 'EXX',
 'SNT',
 'ARI',
 'PIK',
 'MTM',
 'LBH',
 'VVO',
 'TBS',
 'NTC',
 'AVI',
 'DGH',
 'PFG',
 'RES',
 'TRU',
 'DCP',
 'ASR',
 'GTC',
 'BAW',
 'SRE',
 'SAP',
 'ITE',
 'EPP',
 'VKE',
 'MTH',
 'CML',
 'TKG',
 'RBP',
 'MSP',
 'HYP',
 'TCP',
 'FFB',
 'AHA',
 'KST',
 'AFE',
 'RCL',
 'MSM',
 'EQU',
 'IPF',
 'RLO',
 'TSG',
 'IPL',
 'JSE',
 'DRD',
 'RPL',
 'SPG',
 'KAP',
 'AEL',
 'TXT',
 'ARL',
 'ZED',
 'CTK',
 'AIP',
 'STP',
 'ATT',
 'OCE',
 'DTC',
 'EMI',
 'MNK',
 'AFX',
 'HCI',
 'AFH',
 'SNH',
 'FBR',
 'WBO',
 'SAC',
 'SSS',
 'ADH',
 'TTO

In [ ]:
#@title
import pandas as pd
from urllib.request import urlopen
import json
html = urlopen('https://api.stockdio.com/data/financial/prices/v1/GetHistoricalPrices?app-key=A48E1C73560044A3B2F9DE240899D274&stockExchange=JSE&symbol=TFG&from=2020-08-24&to=2022-08-25&useAdjusted=false').read()

data1 = html.decode('utf-8').replace("'", '"')

data = json.loads(data1)
s = json.dumps(data, indent=4)
print(s)

# Writing to sample.json
with open("TFG.json", "w") as outfile:
    outfile.write(s)

In [ ]:
#@title
import pandas as pd
from urllib.request import urlopen
import json
html = urlopen('https://api.stockdio.com/data/financial/info/v1/GetIndexInfo?app-key=55E6EB9A851947438E97EB72DE3F02B3&index=SPX').read()

data1 = html.decode('utf-8')

data = json.loads(data1)
s = json.dumps(data, indent=4)
print(s)

In [ ]:
#@title
with open("naspers.json", "r") as read_it:
     data = json.load(read_it)

print (data['data'])

In [ ]:
#@title
df = pd.read_json('/content/naspers.json')
df.drop(['code', 'message','symbol', 'company','exchange'], inplace = True )

In [ ]:
#@title
df_nested_list = pd.json_normalize(df, record_path = ['data'])

In [ ]:
#@title
# Opening JSON file
with open('naspers.json', 'r') as openfile:
 
    # Reading from json file
    naspers = json.load(openfile)
 
print(naspers)
print(type(naspers))

In [ ]:
#@title
print(naspers.keys())

In [ ]:
#@title
print(naspers["data"].keys())

In [ ]:
#@title
NPN = pd.json_normalize(naspers["data"])
NPN.drop(["symbol", "company", "exchange", "prices.columns"], axis = 1, inplace = True)
NPN

In [ ]:
#@title
out = NPN.explode('prices.values').assign(Co2 = lambda x: x['prices.values'].str.get('prices.values')).reset_index(drop=True)
out.rename(columns = {'prices.values':'prices'}, inplace = True)
out.drop(["Co2"], axis = 1, inplace = True)

In [ ]:
#@title
df2 = pd.DataFrame(out)
df2[['date','open','high','low', 'close', 'volume']] = pd.DataFrame(df2.prices.tolist(), index= df2.index)
df3 = pd.DataFrame(df2['prices'].to_list(), columns=['date','open','high','low', 'close', 'volume'])

# Extract data from API

In [ ]:
import pandas as pd
from urllib.request import urlopen
import json
import requests

results = []
responses = list()
for stock in Stocks:
  res = requests.get('https://api.stockdio.com/data/financial/prices/v1/GetHistoricalPrices?app-key=A48E1C73560044A3B2F9DE240899D274&stockExchange=JSE&symbol={}&from=2022-09-06&to=2022-09-13&useAdjusted=false'.format(stock))
  
  if res.status_code == 200:
    results.append(res.json())
  else:
    print('Request to {} failed'.format(stock))

In [ ]:
results

In [ ]:
import json

# Serializing json
jse_data = json.dumps(results, indent=4)

# Writing to sample.json
with open("jse_data.json", "w") as outfile:
    outfile.write(jse_data)

# Normalize data & convert to Dataframe

In [89]:
# Opening JSON file
import json
import pandas as pd
with open('/content/drive/MyDrive/jse_data.json') as openfile:
 
    # Reading from json file
    jse_data = json.load(openfile)
 
print(jse_data)
print(type(jse_data))

[{'status': {'message': 'Invalid Symbol', 'code': 100}}, {'status': {'code': 0, 'message': ''}, 'data': {'symbol': 'ANH', 'company': 'Anheuser-Busch InBev SA NV', 'exchange': 'SOUTH_AFRICA', 'prices': {'columns': ['date', 'open', 'high', 'low', 'close', 'volume'], 'values': [['2022-09-06T15:00:00Z', 845.14, 849.26, 837.46, 846.0, 345083], ['2022-09-07T15:00:00Z', 838.2, 847.0, 835.37, 845.52, 167821], ['2022-09-08T15:00:00Z', 845.52, 863.79, 845.52, 862.1, 284558], ['2022-09-09T15:00:00Z', 865.42, 868.79, 859.43, 860.52, 225908], ['2022-09-12T15:00:00Z', 860.0, 886.52, 865.13, 881.81, 241388], ['2022-09-13T15:00:00Z', 881.81, 892.34, 873.37, 873.43, 408257]]}}}, {'status': {'message': 'Invalid Symbol', 'code': 100}}, {'status': {'code': 0, 'message': ''}, 'data': {'symbol': 'BTI', 'company': 'British American Tobacco plc', 'exchange': 'SOUTH_AFRICA', 'prices': {'columns': ['date', 'open', 'high', 'low', 'close', 'volume'], 'values': [['2022-09-06T15:00:00Z', 698.5, 696.19, 684.54, 694.

In [90]:
jse_df = pd.json_normalize(jse_data)
jse_new_df = jse_df.drop(["status.message", "status.code", "data.exchange","data.prices.columns" ], axis = 1, inplace = True)
jse_new_df = jse_df.dropna()

In [91]:
jse_new_df

,data.symbol,data.company,data.prices.values
1,ANH,Anheuser-Busch InBev SA NV,"[[2022-09-06T15:00:00Z, 845.14, 849.26, 837.46..."
3,BTI,British American Tobacco plc,"[[2022-09-06T15:00:00Z, 698.5, 696.19, 684.54,..."
4,NPN,Naspers Ltd.,"[[2022-09-06T15:00:00Z, 2399.5, 2417.05, 2355...."
5,CFR,Compagnie Financiere Richemont SA,"[[2022-09-06T15:00:00Z, 185.6, 188.3, 185.15, ..."
6,GLN,Glencore PLC,"[[2022-09-06T15:00:00Z, 93.1, 97.27, 93.1, 96...."
...,...,...,...
246,SPA,Spanjaard Ltd,[]
247,MZR,Mazor Group Ltd,[]
248,ADW,African Dawn Capital Ltd,"[[2022-09-06T15:00:00Z, 0.12, 0.13, 0.12, 0.13..."
249,ACE,Accentuate Ltd,[]


In [ ]:
mydf = jse_new_df.set_index("data.symbol")

for index, data in mydf.iterrows():
    data.to_csv(f'/content/drive/MyDrive/Untitled Folder/{data["data.company"]}.csv')

In [ ]:
jse_new_df = jse_new_df.set_index("data.symbol")

# then loop
for idx in jse_new_df.index:
    jse_new_df.loc[[idx]].to_csv(f'/content/drive/MyDrive/JSE_Stocks_2/{idx}.csv')

In [162]:
A = pd.read_csv('/content/drive/MyDrive/JSE_Stocks_2/AEL.csv', index_col=0)

A.info()

print(A.head(5))

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, AEL to AEL
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   data.company        1 non-null      object
 1   data.prices.values  1 non-null      object
dtypes: object(2)
memory usage: 24.0+ bytes
                             data.company  \
data.symbol                                 
AEL          Allied Electronics Corp Ltd.   

                                            data.prices.values  
data.symbol                                                     
AEL          [['2022-09-06T15:00:00Z', 8.98, 9.04, 8.66, 8....  


In [163]:
#Change 'PriceUSD' columns to 'close'from ast import literal_eval
A.rename(columns = {'data.prices.values':'data_price_values'}, inplace = True)

In [164]:
A['data_price_values'] = A['data_price_values'].apply(literal_eval)
A = A.explode('data_price_values')

In [165]:
A['TimeSeries'] = [','.join(map(str, l)) for l in A['data_price_values']]
Times_Series = A.drop(["data.company","data_price_values",], axis = 1)#, inplace = True)

In [166]:
new = Times_Series["TimeSeries"].str.split(",", n = 6, expand = True)
 
# making separate first name column from new data frame
Times_Series["date"]= new[0]
Times_Series["open"]= new[1]
Times_Series["high"]= new[2]
Times_Series["low"]= new[3]
Times_Series["close"]= new[4]
Times_Series["volume"]= new[5]

# Dropping old Name columns
Times_Series.drop(columns =["TimeSeries"], inplace = True)
 
# df display
Times_Series

,date,open,high,low,close,volume
data.symbol,,,,,,
AEL,2022-09-06T15:00:00Z,8.98,9.04,8.66,8.7,146851
AEL,2022-09-07T15:00:00Z,8.97,8.97,8.48,8.6,179102
AEL,2022-09-08T15:00:00Z,8.75,8.85,8.6,8.85,67140
AEL,2022-09-09T15:00:00Z,8.85,8.88,8.7,8.79,183329
AEL,2022-09-12T15:00:00Z,8.73,9.0,8.73,8.84,888350
AEL,2022-09-13T15:00:00Z,8.85,8.88,8.66,8.7,712353


# **BS, PICKLE & REQUESTS**

In [170]:
import bs4 as bs
import requests
import yfinance as yf
import datetime

resp = requests.get('https://sashares.co.za/shares-list/#gs.g1jxc5')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table')
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]
start = datetime.datetime(2021,7,25)
end = datetime.datetime(2022,8,25)
data = yf.download(tickers, start=start, end=end)
print(data)

[*********************100%***********************]  515 of 515 completed

385 Failed downloads:
- MTU: No data found for this date range, symbol may be delisted
- BHETNQ: No data found for this date range, symbol may be delisted
- STXQUA: No data found for this date range, symbol may be delisted
- SYGWD: No data found for this date range, symbol may be delisted
- ETFGLD: No data found for this date range, symbol may be delisted
- EMH: No data found for this date range, symbol may be delisted
- NEWPRP: No data found for this date range, symbol may be delisted
- PPETNQ: No data found for this date range, symbol may be delisted
- OCE: No data found for this date range, symbol may be delisted
- BIK: No data found for this date range, symbol may be delisted
- SNH: No data found for this date range, symbol may be delisted
- APETNQ: No data found for this date range, symbol may be delisted
- RLO: No data found for this date range, symbol may be delisted
- CVW: No data found for this date rang

In [173]:
print(data)

           Adj Close                                                     \
                 4SI ABETNC ABETNQ         ABG ABSGEA ABSP      ACL ACS   
Date                                                                      
2021-07-26       NaN    NaN    NaN  196.979996    NaN  NaN  11400.0 NaN   
2021-07-27       NaN    NaN    NaN  196.919998    NaN  NaN  11600.0 NaN   
2021-07-28       NaN    NaN    NaN  194.610001    NaN  NaN  11600.0 NaN   
2021-07-29       NaN    NaN    NaN  202.619995    NaN  NaN  11500.0 NaN   
2021-07-30       NaN    NaN    NaN  205.460007    NaN  NaN  11550.0 NaN   
...              ...    ...    ...         ...    ...  ...      ...  ..   
2022-08-18       NaN    NaN    NaN  188.419998    NaN  NaN      NaN NaN   
2022-08-19       NaN    NaN    NaN  186.149994    NaN  NaN      NaN NaN   
2022-08-22       NaN    NaN    NaN  181.429993    NaN  NaN      NaN NaN   
2022-08-23       NaN    NaN    NaN  181.839996    NaN  NaN      NaN NaN   
2022-08-24       NaN    N

In [172]:
tickers

['4SI',
 'ABETNC',
 'ABETNQ',
 'ABG',
 'ABSGEA',
 'ABSP',
 'ACL',
 'ACS',
 'ACT',
 'ADETNC',
 'ADETNQ',
 'ADH',
 'ADR',
 'ADW',
 'AEE',
 'AEG',
 'AEL',
 'AFE',
 'AFEP',
 'AFH',
 'AFT',
 'AGL',
 'AHL',
 'AIL',
 'AIP',
 'ALETNC',
 'ALETNQ',
 'AME',
 'AMETNC',
 'AMETNQ',
 'AMIB50',
 'AMS',
 'ANG',
 'ANH',
 'ANI',
 'AON',
 'AOO',
 'AOVP',
 'APETNC',
 'APETNQ',
 'APF',
 'APH',
 'APN',
 'APO',
 'ARA',
 'ARI',
 'ARL',
 'ART',
 'ASBI',
 'ASC',
 'ATI',
 'ATID',
 'ATIG',
 'ATT',
 'AVI',
 'AVL',
 'AVV',
 'AYO',
 'BAT',
 'BAW',
 'BAWP',
 'BCF',
 'BEL',
 'BHETNC',
 'BHETNQ',
 'BHG',
 'BID',
 'BIHLEB',
 'BIK',
 'BKI',
 'BLU',
 'BRN',
 'BRT',
 'BSR',
 'BTI',
 'BUC',
 'BVT',
 'BWN',
 'BYI',
 'CAA',
 'CAC',
 'CAT',
 'CATP',
 'CCO',
 'CFR',
 'CGN',
 'CGR',
 'CHP',
 'CKS',
 'CLH',
 'CLI',
 'CLS',
 'CMH',
 'CML',
 'CMO',
 'CND',
 'COETNC',
 'COETNQ',
 'COH',
 'CPI',
 'CPIP',
 'CRP',
 'CSB',
 'CSP500',
 'CSPROP',
 'CTA',
 'CTOP50',
 'CVW',
 'DCCUS2',
 'DCCUSD',
 'DCP',
 'DGH',
 'DIB',
 'DIVTRX',
 'DKR',
 '

In [ ]:
!pip install pandas_datareader

In [175]:

# pip install yfinance
import yfinance as yf
import datetime as datetime
 
# Time starts from here
start = "2020-10-12"
end = "2022-10-12" 
 
ticker_list = tickers
 
# Here we use yf.download function
data = yf.download(
    tickers=ticker_list,
    threads=True,
    group_by='ticker',
    start=start, 
    end=end
 
)
 
# used for making transpose
data = data.T
 
for t in ticker_list:
    # printing name
    print(t)
    print('\n')
     
    # used data.loc as it takes only index
    # labels and returns dataframe
    print(data.loc[t]) 
    print('\n')
# Total time calculated
print('The program takes ', time.time()-start, 'seconds.')

Streaming output truncated to the last 5000 lines.
Close            5.35        5.00        5.04        5.14        5.85   
Adj Close        5.35        5.00        5.04        5.14        5.85   
Volume      542000.00   897000.00   405100.00   447300.00  1061500.00   

Date       2022-10-05  2022-10-06  2022-10-07  2022-10-10  2022-10-11  
Open             5.71        5.32        5.01        4.91        4.58  
High             5.77        5.50        5.06        4.91        4.67  
Low              5.27        5.04        4.85        4.51        4.38  
Close            5.33        5.07        4.90        4.54        4.59  
Adj Close        5.33        5.07        4.90        4.54        4.59  
Volume      733400.00   774700.00   898500.00   767400.00   639700.00  

[6 rows x 504 columns]


SOLBE1


Date       2020-10-12  2020-10-13  2020-10-14  2020-10-15  2020-10-16  \
Open              NaN         NaN         NaN         NaN         NaN   
High              NaN         NaN         Na

KeyError: ignored

In [ ]:
data.dropna(inplace = True)

In [ ]:
data